In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split


In [13]:
npz = np.load('/Users/shahzadiaiman/.cache/kagglehub/datasets/saurabhbagchi/ship-and-iceberg-images/versions/1/input_data.npz')

X_train = npz['X_train']
Y_train = npz['Y_train']
del npz

print('We have {} examples to work with'.format(Y_train.shape[0]))

We have 4113 examples to work with


In [14]:
# Normalize
X_train = X_train.astype(np.float32)
X_train = (X_train - X_train.mean()) / X_train.std()

# Split into train/validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train
)

In [15]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling2D((2,2)),
    BatchNormalization(),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    BatchNormalization(),

    GlobalAveragePooling2D(),

    Dense(256, activation='relu'),
    Dropout(0.4),

    Dense(1, activation='sigmoid')  # binary classification: ship vs iceberg
])


/Users/shahzadiaiman/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Compile
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [17]:
#callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)


In [20]:
# Train
history = model.fit(
    X_train, y_train,                    # raw training data
    validation_data=(X_val, y_val),      # validation set
    epochs=50,                            # can run longer; early stopping will stop if needed
    batch_size=32,                        # smaller batch helps generalization
    callbacks=[early_stop, reduce_lr]     # pass the callbacks here
)

Epoch 1/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 8s 68ms/step - accuracy: 0.6798 - loss: 0.6159 - val_accuracy: 0.6877 - val_loss: 0.6367 - learning_rate: 1.0000e-04
Epoch 2/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - accuracy: 0.7473 - loss: 0.5396 - val_accuracy: 0.7849 - val_loss: 0.5627 - learning_rate: 1.0000e-04
Epoch 3/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.7736 - loss: 0.4816 - val_accuracy: 0.7922 - val_loss: 0.4755 - learning_rate: 1.0000e-04
Epoch 4/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - accuracy: 0.7769 - loss: 0.4565 - val_accuracy: 0.8092 - val_loss: 0.4199 - learning_rate: 1.0000e-04
Epoch 5/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accuracy: 0.7805 - loss: 0.4239 - val_accuracy: 0.8141 - val_loss: 0.3840 - learning_rate: 1.0000e-04
Epoch 6/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 7s 72ms/step - accuracy: 0.7967 - loss: 0.3936 - val_accuracy: 0.8190 - val_loss: 0.3705 - learning_rate: 1.0000e-04
Epoch 7/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 7s 72ms/step - acc

In [21]:
loss, acc = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {acc:.4f}")

# Get predictions
y_pred_probs = model.predict(X_val)
y_pred = (y_pred_probs > 0.5).astype("int32")

# Compute recall
recall = recall_score(y_val, y_pred)
print(f"Validation Recall: {recall:.4f}")


Validation Accuracy: 0.8858
26/26 ━━━━━━━━━━━━━━━━━━━━ 24s 954ms/step
Validation Recall: 0.8889
